# Pandas Data Engineering

### Python for Data Engineering

Exercise:

Read the titanic csv file in and save as a pandas data frame called ti.

## How do I select rows by index?

Exercise:

Use the .loc\[\] method to select the 'sex' column, 1st, 4th, and 6th rows.

Hint: Recall that the locator method `.loc` accepts *row indexes* and column *names*

## How do I select rows by position?

As `.loc` is an index-based method, it does not select elements by position. However `.iloc` selects **both** rows and columns by position.

Exercise:

Use.iloc to select the survived and sex columns, 1st and 2nd row.

## How do I create new columns?

To create a new column "select" it using `['colname']` and assign using `=`

Exercise

1. Select the sex column and use .str.upper() to convert all entries to upper case. Save this as ti\['new_column'\].

2. Select your new column from ti and use .sample() or .head() to check your previous answer.

All columns must have the same number of rows; since new columns are *mostly* just derived from existing, this may not be a common issue. 

## How do I group and aggregate data?

```
df['column-to-be-aggregated'].groupby(
    df['values-to-split-on']
).summaryfn()
```

* `SELECT age FROM df GROUP BY survived`
* `SELECT age FROM df GROUP BY survived, class`
* `SELECT age FROM df WHERE age <= 18 GROUP BY survived, class`

What is the average age of the Titanic passengers?

Exercise:

1. Select the fare column.

2. Extend your previous answer by using .groupby() with the survived column as the values to split on. Use .mean() as the summary function for each group.

3. Select the age column and group into a set of rows for the survivors and a set of rows for those who did not survive. Calculate the mean age for each group.

## How I groupby multiple columns?

As in most cases in pandas a single column can be replaced with a list of them:

Exercise:

Run the below section of code and take a moment to think about how the results were calculated.

In [10]:
columns = [ti['survived'], ti['class']]
ti['fare'].groupby(columns).mean()


survived  class 
0         First     64.684008
          Second    19.412328
          Third     13.669364
1         First     95.608029
          Second    22.055700
          Third     13.694887
Name: fare, dtype: float64

## How do I create new factor/group columns?

The `pd.cut` method on the pandas library *cuts* a column (ie., bins, groups) using *cut points*...

Exercise:

1. Run the below section of code and then adapt it so that the age_groups column is saved as a new column in the ti dataframe with a different name: 'Age_Groups'.

2. Select the age and age_groups columns

3. Extend your previous answer by using .loc to view the rows in which age is 17 - which label was applied for 17 year olds when pd.cut() was used?


In [6]:
age_groups = pd.cut(
    ti['age'], 
    [0, 17, 30, 50, 70, 100], 
    labels=["child", "yadult", "adult", "senior", "vsnr"]
)

## How do I use columns that aren't part of the dataframe?

You can use the new column in `groupby` directly:

In [27]:
columns = [ ti['embark_town'], age_groups ]

ti['survived'].groupby(columns).mean()

embark_town  age   
Cherbourg    child     0.750000
             yadult    0.580000
             adult     0.585366
             senior    0.615385
             vsnr      0.000000
Queenstown   child     0.428571
             yadult    0.444444
             adult     0.111111
             senior    0.000000
             vsnr      0.000000
Southampton  child     0.487805
             yadult    0.303797
             adult     0.400000
             senior    0.279070
             vsnr      0.500000
Name: survived, dtype: float64

Or we can add this back into the dataframe:

In [28]:
ti['ageg'] = age_groups

Aside: the `range` function provides a handy way of specifying groups,

In [35]:
ti['survived'].groupby(pd.cut(ti['fare'], range(0, 300, 50))).mean()

fare
(0, 50]       0.324022
(50, 100]     0.654206
(100, 150]    0.791667
(150, 200]    0.666667
(200, 250]    0.636364
Name: survived, dtype: float64

## How do I import data from an excel file?

The `read_excel` function imports data:

Exercise:

Use pd.read_excel() to read from the file titanic.xlsx . Save this dataframe as raw.

## How do I select different excel sheets?

Exercise:

Open the .xlsx file in Excel.

Run the below section of code and check what data has been read into the pandas dataframe.

In [63]:
sheet1 = pd.read_excel('titanic.xlsx',  sheet_name=0)

## How do I rename columns?

In [11]:
new = ti.rename(columns={
    'survived': 'P_Survival',
    'age': 'X_Age',
    'sex': 'X_Sex'
})

## How do I remove columns?

Columns can be "removed" simply by selecting a subset of the ones you wish to keep:

In [13]:
selected = ['P_Survival', 'X_Age', 'X_Sex']

new = new[selected]
new.sample()

,P_Survival,X_Age,X_Sex
50,0,7.0,male


Or by `drop`ing,

In [20]:
smaller = ti.copy()
smaller.drop(columns=['pclass', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone']).sample()

,survived,sex,age
26,0,male,NaN


## How do I change the type of a column?

The `astype` function *converts*, which requires a sensible conversion to exist:

In [27]:
ti['y'] = ti['survived'].astype(bool)

In [28]:
ti['y'].sample()

771    False
Name: y, dtype: bool

This will not work with text as it isnt clear, eg., that "male" should be `True` (or `False`). The default for text to boolean is: if there is any text it will save it as True, and if it's an empty string it will save it as False (not very helpful in this case).

Here then, just use a test:

In [32]:
ti['sex_cat'] = ti['sex'] == "male"

ti['sex_cat'].sample()

623    True
Name: sex_cat, dtype: bool

## How do I join datasets?

* `pd.concat`
    * gluing at the end using index position
* `df.merge`
    * sql-like
    * joining columns togther
    * based on column which overlaps

In [28]:
df1 = pd.DataFrame({'location': ['london', 'leeds', 'manchester'],
               'value': [1, 2, 3]}
)
df2 = pd.DataFrame({'city': ['london', 'leeds', 'manchester'],
                     'value': [ 6, 7, 8]})

df1.merge(df2, left_on='location', right_on='city')

,location,value_x,city,value_y
0,london,1,london,6
1,leeds,2,leeds,7
2,manchester,3,manchester,8


## Appendix

Download and save the files needed for this module:

In [3]:
import seaborn as sns
sns.load_dataset('titanic').to_excel('titanic.xlsx', index=False)
sns.load_dataset('tips').to_csv('tips.csv', index=False)
sns.load_dataset('titanic').to_csv('titanic.csv', index=False)

## How do I create a column using a conditional business rule?

The `numpy` library provides additional tools that are useful with pandas, eg., the `.select` method runs a test *over a column* to select values:

In [5]:
import numpy as np
np.select([ True, False], ["YES", "NO" ])

array('YES', dtype='<U21')

In [8]:
import numpy as np

conditions =  [ti['sex'] == 'male', ti['sex'] != 'male']
options    =  [ti['age'], ti['fare']]

ti['measure'] = np.select(conditions, options)
ti['measure'].sample()

493    71.0
Name: measure, dtype: float64

## Exercise

* show the first ten rows of:
    * the first ten cols
    * the last ten cols
    * see 'How do I select rows by position?'
    
* add a column to the dataframe `is_adult` which tests the `age` col
    * see 'How do I create new columns?'
    
* rename the `pclass` column to `passgener_class`
    * see 'How do I rename columns?'
    

    
## Exercise  
* Consider the following groupby which uses `agg` to run several summarizing operations...

```python
cols = ti['sex']
ti['age'].groupby(cols).agg(['mean', 'median', 'count'])
```

* with this as a tempalte, report similar measures for:
* survived by sex
* survived by embark_town
* survived by adult_male

* Consider the following code which defines groups then uses them in a groupby...

```python
groups = [0, 12, 18, 30, 65, 100]
names = ["Child", "Teen", "YAdult", "Adult", "Retired"]

age_groups = pd.cut( ti['age'], groups, labels=names )
ti['survived'].groupby(age_groups).mean()
```

* with this code as a template,
* the rate of survival for different fare groups
* eg., 0 to 10, 10 to 50, 50 to 100, 100 to 1000, etc.